# Langchain crash course

In [19]:
# from secret_key import openapi_key
# import os
# os.environ['OPENAI_API_KEY'] = openapi_key

import os
os.environ['OPENAI_API_KEY'] = "add your key here"

In [20]:
pip install -U langchain langchain-openai openai

Note: you may need to restart the kernel to use updated packages.


## LLMs

In [22]:
# secret_key.py should contain: OPENAI_API_KEY = "sk-...."
from secret_key import OPENAI_API_KEY
from langchain_openai import ChatOpenAI
import os

os.environ["OPENAI_API_KEY"] = OPENAI_API_KEY.strip()  # strip accidental spaces/newlines

llm = ChatOpenAI(model="gpt-4o-mini", temperature=0.9)
resp = llm.invoke("I want to open a restaurant for Indian food. Suggest a fancy name.")
print(resp.content)


Here are some fancy name suggestions for your Indian restaurant:

1. **Spice Symphony**
2. **Curry & Canvas**
3. **Masala Majestic**
4. **Saffron Serenade**
5. **The Royal Tandoor**
6. **Bollywood Bites**
7. **Culinary Maharaja**
8. **Naan & Nectar**
9. **Mystic Masala**
10. **Gastronomic India**

Feel free to mix and match or modify these suggestions to fit your vision!


In [25]:
pip install -U langchain-core

Note: you may need to restart the kernel to use updated packages.


## Prompt Templates

In [26]:
from langchain_core.prompts import PromptTemplate

prompt_template_name = PromptTemplate(
    input_variables =['cuisine'],
    template = "I want to open a restaurant for {cuisine} food. Suggest a fency name for this."
)
p = prompt_template_name.format(cuisine="Italian")
print(p)

# print(llm.predict(p))

I want to open a restaurant for Italian food. Suggest a fency name for this.


In [38]:
pip install -U langchain langchain-core langchain-openai langchain-community

Note: you may need to restart the kernel to use updated packages.


## Chains

In [40]:
from langchain_openai import ChatOpenAI
from langchain_core.prompts import PromptTemplate

# Define model
llm = ChatOpenAI(model="gpt-4o-mini", temperature=0.9)

# Define prompt
prompt = PromptTemplate(
    input_variables=["cuisine"],
    template="I want to open a restaurant for {cuisine} food. Suggest a fancy name for this."
)

# Build the chain using the new 'Runnable' syntax
chain = prompt | llm

# Run it
response = chain.invoke({"cuisine": "Mexican"})
print(response.content)


Here are some fancy name suggestions for your Mexican restaurant:

1. **Casa de Sabores** (House of Flavors)
2. **El Sabor Secreto** (The Secret Flavor)
3. **Agave & Aroma**
4. **La Fiesta Gourmet**
5. **Sazón y Estilo** (Seasoning and Style)
6. **Cielo Mexicano** (Mexican Heaven)
7. **Estrella de México** (Star of Mexico)
8. **La Palapa Prestige**
9. **Rincón Mexicano** (Mexican Corner)
10. **Tierra de Encanto** (Land of Enchantment)

Feel free to mix and match or modify them to better fit your vision!


In [47]:
# pip installs (run once in a cell/terminal if needed)
# pip install -U langchain langchain-core langchain-openai

from langchain_openai import ChatOpenAI
from langchain_core.prompts import PromptTemplate
from langchain_core.output_parsers import StrOutputParser

import os
# make sure your key is set; replace with your import or .env loader as you prefer
# from secret_key import OPENAI_API_KEY
# os.environ["OPENAI_API_KEY"] = OPENAI_API_KEY.strip()

llm = ChatOpenAI(model="gpt-4o-mini", temperature=0.6)

# 1) Restaurant name prompt
prompt_name = PromptTemplate(
    input_variables=["cuisine"],
    template="I want to open a restaurant for {cuisine} food. Suggest a short, catchy, fancy name."
)

# 2) Menu items prompt
prompt_items = PromptTemplate(
    input_variables=["restaurant_name"],
    template="Suggest 8 signature menu items for a restaurant named '{restaurant_name}'."
)

# Parse AIMessage -> string
to_text = StrOutputParser()

# Chain A: cuisine -> restaurant_name (string)
name_chain = prompt_name | llm | to_text

# Chain B: take restaurant_name (from Chain A) -> items (string)
# We build a dict where 'restaurant_name' is produced by name_chain
menu_chain = {"restaurant_name": name_chain} | prompt_items | llm | to_text

# Run the whole thing
result = menu_chain.invoke({"cuisine": "Mexican"})
print(result)


Here are eight signature menu items for "Casa de Sazón" that emphasize the warmth and richness of Mexican cuisine while creating a homey atmosphere:

1. **Sazón Tacos al Pastor**  
   Marinated pork cooked on a vertical spit, served in warm corn tortillas with fresh pineapple, cilantro, and onion. Accompanied by a side of homemade salsa verde and pickled red onions.

2. **Cazuela de Frijoles Negros**  
   A comforting black bean stew simmered with garlic, onion, and spices, topped with crumbled queso fresco and served with warm corn tortillas for dipping.

3. **Mole Poblano Enchiladas**  
   Corn tortillas filled with shredded chicken and smothered in a rich, homemade mole sauce made with chocolate, chiles, and spices. Served with a side of Mexican rice and refried beans.

4. **Sazón Pozole Rojo**  
   A hearty hominy soup with tender pork, seasoned with guajillo chiles and garnished with radishes, cabbage, and lime. Served with a side of crispy tortilla chips.

5. **Chiles Rellenos de

#### Simple Sequential Chain

In [49]:
# Install or upgrade first if not done
# pip install -U langchain langchain-core langchain-openai

from langchain_openai import ChatOpenAI
from langchain_core.prompts import PromptTemplate
from langchain_core.output_parsers import StrOutputParser

# Initialize LLM
llm = ChatOpenAI(model="gpt-4o-mini", temperature=0.7)

# 1️⃣ Step 1 - Restaurant Name Generator
prompt_name = PromptTemplate(
    input_variables=["cuisine"],
    template="I want to open a restaurant for {cuisine} food. Suggest a fancy, catchy name."
)
name_chain = prompt_name | llm | StrOutputParser()

# 2️⃣ Step 2 - Menu Item Generator (takes restaurant name)
prompt_menu = PromptTemplate(
    input_variables=["restaurant_name"],
    template="Suggest 5 signature dishes for the restaurant named '{restaurant_name}'."
)
menu_chain = prompt_menu | llm | StrOutputParser()

# 3️⃣ Combine both chains (sequentially)
# The output of name_chain feeds into menu_chain automatically
full_chain = {"restaurant_name": name_chain} | menu_chain

# Run the full chain
result = full_chain.invoke({"cuisine": "Indian"})
print(result)


Here are five signature dishes that would complement the fancy and catchy names of your Indian restaurant:

1. **Royal Saffron Biryani**: A fragrant basmati rice dish layered with marinated chicken or lamb, slow-cooked with saffron, whole spices, and caramelized onions, served with a side of cooling raita.

2. **Masala Lobster Tikka**: Tender lobster marinated in a rich blend of spices and yogurt, grilled to perfection in a tandoor, and served with a tangy mint chutney and pickled onions.

3. **Butter Chicken Couture**: A luxurious twist on the classic butter chicken, featuring succulent pieces of chicken simmered in a creamy, spiced tomato sauce, garnished with fresh cilantro and served with garlic naan.

4. **Paneer Tikka Platter**: A vibrant assortment of marinated paneer cubes and seasonal vegetables, skewered and grilled, served alongside a trio of house-made chutneys – mint, tamarind, and mango.

5. **Chaat Extravaganza**: A delightful fusion of traditional Indian street food fea

#### Sequential Chain

In [51]:
# pip install -U langchain langchain-core langchain-openai

from langchain_openai import ChatOpenAI
from langchain_core.prompts import PromptTemplate
from langchain_core.output_parsers import StrOutputParser
import os

# If you load the key from your module or .env, set it here
# from secret_key import OPENAI_API_KEY
# os.environ["OPENAI_API_KEY"] = OPENAI_API_KEY.strip()

llm = ChatOpenAI(model="gpt-4o-mini", temperature=0.7)

prompt_template_name = PromptTemplate(
    input_variables=['cuisine'],
    template="I want to open a restaurant for {cuisine} food. Suggest a fancy name for this."
)

# prompt -> llm -> string
chain = prompt_template_name | llm | StrOutputParser()

print(chain.invoke({"cuisine": "Indian"}))


Here are some fancy name suggestions for your Indian restaurant:

1. **Saffron Elegance**
2. **Masala Majestic**
3. **Spice Symphony**
4. **Nirvana Indian Kitchen**
5. **Curry Couture**
6. **Tandoor Treasures**
7. **Bollywood Bites**
8. **Karma Cuisine**
9. **The Royal Thali**
10. **Chutney & Charm**

Feel free to mix and match or modify any of these suggestions to find the perfect name for your restaurant!


In [53]:
# Make sure you have these installed
# pip install -U langchain langchain-core langchain-openai

from langchain_openai import ChatOpenAI
from langchain_core.prompts import PromptTemplate
from langchain_core.output_parsers import StrOutputParser

# Initialize model
llm = ChatOpenAI(model="gpt-4o-mini", temperature=0.7)

# Define prompt
prompt_template_items = PromptTemplate(
    input_variables=["restaurant_name"],
    template="Suggest some menu items for {restaurant_name}."
)

# New runnable-style chain (replaces LLMChain)
food_items_chain = prompt_template_items | llm | StrOutputParser()

# Run the chain
result = food_items_chain.invoke({"restaurant_name": "Spice Symphony"})
print(result)


Spice Symphony is known for its vibrant blend of traditional and contemporary Indian cuisine. Here are some menu item suggestions that would fit well within that theme:

### Starters
1. **Tandoori Cauliflower Bites** - Marinated cauliflower florets grilled in tandoor with a spicy yogurt sauce.
2. **Paneer Tikka Skewers** - Grilled marinated paneer served with mint chutney and pickled onions.
3. **Spicy Lamb Seekh Kebabs** - Ground lamb mixed with spices, grilled on skewers, served with a tangy tamarind sauce.
4. **Vegetable Pakoras** - Assorted vegetables dipped in spiced chickpea batter and fried until crispy, served with green chutney.
5. **Prawn Puri** - Crispy puris topped with spiced prawns and a drizzle of tamarind sauce.

### Soups and Salads
6. **Tomato Shorba** - A spiced tomato soup garnished with fresh cilantro and served with crispy naan bread.
7. **Chickpea and Quinoa Salad** - A refreshing salad with chickpeas, quinoa, cucumber, and a zesty lemon dressing.

### Main Cours

In [55]:
# Install or update to the latest LangChain packages if needed
# pip install -U langchain langchain-core langchain-openai

from langchain_openai import ChatOpenAI
from langchain_core.prompts import PromptTemplate
from langchain_core.output_parsers import StrOutputParser

# Step 1️⃣ — Initialize model
llm = ChatOpenAI(model="gpt-4o-mini", temperature=0.7)

# Step 2️⃣ — Define prompt for restaurant name
prompt_name = PromptTemplate(
    input_variables=["cuisine"],
    template="I want to open a restaurant for {cuisine} food. Suggest a fancy restaurant name."
)

# Step 3️⃣ — Define prompt for menu items
prompt_items = PromptTemplate(
    input_variables=["restaurant_name"],
    template="Suggest 5 unique menu items for a restaurant named '{restaurant_name}'."
)

# Step 4️⃣ — Convert each step into a chain (Runnable)
name_chain = prompt_name | llm | StrOutputParser()
menu_chain = prompt_items | llm | StrOutputParser()

# Step 5️⃣ — Sequential pipeline (replaces SequentialChain)
# The output of name_chain feeds automatically into menu_chain
full_chain = {"restaurant_name": name_chain} | menu_chain

# Step 6️⃣ — Run the pipeline
result = full_chain.invoke({"cuisine": "Indian"})
print(result)


Here are five unique menu items that would fit beautifully within the theme of your Indian restaurant, inspired by the fancy name ideas provided:

1. **Saffron Symphony Biryani**  
   A luxurious saffron-infused basmati biryani featuring tender marinated lamb or chicken, slow-cooked with caramelized onions, fragrant spices, and served with a side of cooling raita.

2. **Curry Couture Tasting Platter**  
   An elegant presentation of mini curry cups featuring a selection of gourmet curries such as butter chicken, paneer tikka masala, and coconut shrimp curry, accompanied by artisanal naan and jasmine rice.

3. **Masala Mirage Appetizer Trio**  
   A visually stunning trio of appetizers: crispy samosas filled with spiced potatoes, tangy tamarind chutney; tandoori mushroom skewers marinated in yogurt and spices; and delicate vegetable pakoras served with mint chutney.

4. **Tandoor & Truffle Lamb Chops**  
   Grilled lamb chops marinated in a blend of Indian spices and finished with a dri

## Agents

In [14]:
# make sure yoy have installed this package: pip install google-search-results
# from secret_key import serpapi_key
# os.environ['SERPAPI_API_KEY'] = serpapi_key

os.environ['SERPAPI_API_KEY'] = "add your serpapi key here"

#### serpapi and llm-math tool

In [57]:
from langchain.agents import AgentType, initialize_agent, load_tools
from langchain.llms import OpenAI

llm = OpenAI(temperature=0)

# The tools we'll give the Agent access to. Note that the 'llm-math' tool uses an LLM, so we need to pass that in.
tools = load_tools(["serpapi", "llm-math"], llm=llm)

# Finally, let's initialize an agent with the tools, the language model, and the type of agent we want to use.
agent = initialize_agent(tools, llm, agent=AgentType.ZERO_SHOT_REACT_DESCRIPTION, verbose=True)

# Let's test it out!
agent.run("What was the GDP of US in 2022 plus 5?")

ImportError: cannot import name 'AgentType' from 'langchain.agents' (C:\Sidhu\Tech\AI\code\genAI\.venv\Lib\site-packages\langchain\agents\__init__.py)

#### Wikipedia and llm-math tool

In [16]:
# install this package: pip install wikipedia

# The tools we'll give the Agent access to. Note that the 'llm-math' tool uses an LLM, so we need to pass that in.
tools = load_tools(["wikipedia", "llm-math"], llm=llm)

# Finally, let's initialize an agent with the tools, the language model, and the type of agent we want to use.
agent = initialize_agent(
    tools, 
    llm, 
    agent=AgentType.ZERO_SHOT_REACT_DESCRIPTION, 
    verbose=True
)

# Let's test it out!
agent.run("When was Elon musk born? What is his age right now in 2023?")



> Entering new  chain...
 I need to find out when Elon Musk was born and then calculate his age.
Action: Wikipedia
Action Input: Elon Musk
Observation: Page: Elon Musk
Summary: Elon Reeve Musk ( EE-lon; born June 28, 1971) is a business magnate and investor. He is the founder, CEO, and chief engineer of SpaceX; angel investor, CEO, and product architect of Tesla, Inc.; owner, CTO, and chairman of Twitter; founder of the Boring Company and X Corp.; co-founder of Neuralink and OpenAI; and president of the philanthropic Musk Foundation. Musk is the wealthiest person in the world according to the Bloomberg Billionaires Index and Forbes's Real Time Billionaires list as of June 2023, primarily from his ownership stakes in Tesla and SpaceX, with an estimated net worth of around $225 billion according to Bloomberg and $230.2 billion according to Forbes.Musk was born in Pretoria, South Africa, and briefly attended the University of Pretoria before moving to Canada at age 18, acquiring citizen

'Elon Musk was born on June 28, 1971 and is currently 52 years old in 2023.'

## Memory

In [17]:
chain = LLMChain(llm=llm,prompt=prompt_template_name)
name = chain.run("Mexican")
print(name)



Taco Fiesta


In [18]:
name = chain.run("Indian")
print(name)



Maharaja's Palace Cuisine


In [19]:
chain.memory

In [20]:
type(chain.memory)

NoneType

#### ConversationBufferMemory

In [21]:
from langchain.memory import ConversationBufferMemory

memory = ConversationBufferMemory()

chain = LLMChain(llm=llm, prompt=prompt_template_name, memory=memory)
name = chain.run("Mexican")
print(name)



Taco Fiesta


In [22]:
name = chain.run("Arabic")
print(name)



Al-Fez Restaurant


In [23]:
print(chain.memory.buffer)

Human: Mexican
AI: 

Taco Fiesta
Human: Arabic
AI: 

Al-Fez Restaurant


#### ConversationChain

In [24]:
from langchain.chains import ConversationChain

convo = ConversationChain(llm=OpenAI(temperature=0.7))
print(convo.prompt.template)

The following is a friendly conversation between a human and an AI. The AI is talkative and provides lots of specific details from its context. If the AI does not know the answer to a question, it truthfully says it does not know.

Current conversation:
{history}
Human: {input}
AI:


In [25]:
convo.run("Who won the first cricket world cup?")

' The first Cricket World Cup was won by the West Indies in 1975.'

In [26]:
convo.run("How much is 5+5?")

' 10.'

In [27]:
convo.run("Who was the captain ofthe winning team?")

' The captain of the West Indies team that won the first Cricket World Cup in 1975 was Clive Lloyd.'

In [29]:
print(convo.memory.buffer)

Human: Who won the first cricket world cup?
AI:  The first Cricket World Cup was won by the West Indies in 1975.
Human: How much is 5+5?
AI:  10.
Human: Who was the captain ofthe winning team?
AI:  The captain of the West Indies team that won the first Cricket World Cup in 1975 was Clive Lloyd.


#### ConversationBufferWindowMemory

In [30]:
from langchain.memory import ConversationBufferWindowMemory

memory = ConversationBufferWindowMemory(k=1)

convo = ConversationChain(
    llm=OpenAI(temperature=0.7),
    memory=memory
)
convo.run("Who won the first cricket world cup?")

' The first Cricket World Cup was held in 1975 and the winner was the West Indies.'

In [31]:
convo.run("How much is 5+5?")

' 5+5 is 10.'

In [32]:
convo.run("Who was the captain of the winning team?")

" I'm sorry, I don't know."